# Notebook 9: Classical vs Neural Comparison

**Purpose**: Compare classical ML models with neural network models.

**Inputs**:
- `classical_metrics.csv`
- `neural_metrics.csv`

**Outputs**:
- Comparison visualizations → `figures/`

---

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Paths
BASE_DIR = Path('.').resolve().parent
RESULTS_DIR = BASE_DIR / 'results'
FIGURES_DIR = BASE_DIR / 'figures'

# Style
plt.style.use('seaborn-v0_8-whitegrid')
colors = {'classical': '#3498db', 'neural': '#e74c3c'}

In [ ]:
# Load metrics
classical_df = pd.read_csv(RESULTS_DIR / 'classical_metrics.csv')
neural_df = pd.read_csv(RESULTS_DIR / 'neural_metrics.csv')

classical_df['category'] = 'Classical'
neural_df['category'] = 'Neural'

combined_df = pd.concat([classical_df, neural_df], ignore_index=True)

print(f"Classical models: {len(classical_df)}")
print(f"Neural models: {len(neural_df)}")

In [ ]:
# Display all results
print("\nClassical Models:")
print(classical_df[['model', 'f1_score', 'roc_auc', 'train_time']].to_string(index=False))
print("\nNeural Models:")
print(neural_df[['model', 'f1_score', 'roc_auc', 'train_time']].to_string(index=False))

## Metric Comparison Charts

In [ ]:
# Bar chart comparison for each metric
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    
    # Sort by metric value
    data = combined_df.sort_values(metric, ascending=True)
    
    colors_list = [colors['classical'] if c == 'Classical' else colors['neural'] 
                   for c in data['category']]
    
    ax.barh(data['model'], data[metric], color=colors_list)
    ax.set_xlabel(metric.replace('_', ' ').title())
    ax.set_title(f'{metric.replace("_", " ").title()} Comparison')
    ax.set_xlim([0, 1])

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors['classical'], label='Classical'),
                   Patch(facecolor=colors['neural'], label='Neural')]
axes[-1].legend(handles=legend_elements, loc='center')
axes[-1].axis('off')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_neural_metrics.png', dpi=150)
plt.show()

In [ ]:
# Training time comparison
fig, ax = plt.subplots(figsize=(12, 6))

data = combined_df.sort_values('train_time', ascending=True)
colors_list = [colors['classical'] if c == 'Classical' else colors['neural'] 
               for c in data['category']]

ax.barh(data['model'], data['train_time'], color=colors_list)
ax.set_xlabel('Training Time (seconds)')
ax.set_title('Training Time: Classical vs Neural')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_neural_time.png', dpi=150)
plt.show()

In [ ]:
# Box plot comparison by category
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, metric in enumerate(['f1_score', 'roc_auc', 'train_time']):
    ax = axes[i]
    combined_df.boxplot(column=metric, by='category', ax=ax)
    ax.set_title(metric.replace('_', ' ').title())
    ax.set_xlabel('')
    
plt.suptitle('Distribution Comparison: Classical vs Neural', y=1.02)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'classical_vs_neural_boxplot.png', dpi=150)
plt.show()

In [ ]:
# Summary statistics
print("\n" + "="*60)
print("CLASSICAL VS NEURAL SUMMARY")
print("="*60)

for metric in ['f1_score', 'roc_auc', 'train_time']:
    classical_mean = classical_df[metric].mean()
    neural_mean = neural_df[metric].mean()
    
    print(f"\n{metric.replace('_', ' ').title()}:")
    print(f"  Classical: {classical_mean:.4f} (±{classical_df[metric].std():.4f})")
    print(f"  Neural: {neural_mean:.4f} (±{neural_df[metric].std():.4f})")
    
    if metric != 'train_time':
        winner = 'Classical' if classical_mean > neural_mean else 'Neural'
    else:
        winner = 'Classical' if classical_mean < neural_mean else 'Neural'
    print(f"  Winner: {winner}")

print("\n✅ Notebook 9 Complete!")